In [28]:
from base import get_predefined_system, Force, Energy
import sympy as sp
import numpy as np

Sph = get_predefined_system('InvSpherical')


In [29]:
b = sp.symbols('b', nonnegative=True)

damping_force = [-b*unit_v for unit_v in Sph.velocity_vector]
damping_force[1]

-b*r(t)*Derivative(θ(t), t)

In [30]:
### GPE example
m, g, z0 = sp.symbols('m g z0')
h = z0 + Sph.position_vector.dot(Sph.C.k)
E = m*g*h

vector = []
for u in Sph.U:
    vector.append(-sp.diff(E, u))
print(vector)

print(-np.vectorize(sp.diff)(E, Sph.U))

[g*m*cos(θ(t)), -g*m*r(t)*sin(θ(t)), 0]
[g*m*cos(θ(t)) -g*m*r(t)*sin(θ(t)) 0]


In [31]:
gpe = Energy(E, Sph)
for x in gpe.to_force().F:
    display(x)

g*m*cos(θ(t))

-g*m*sin(θ(t))

0

In [32]:
type(-np.vectorize(sp.diff)(E, Sph.U))

numpy.ndarray

In [33]:
from pendulum_forces import PendulumForces
p = PendulumForces(Sph)
p.gravitational_potential_energy().to_force().F

array([g*m*cos(θ(t)), -g*m*sin(θ(t)), 0], dtype=object)

In [34]:
num_magnets = 4
sp.symbols(f'\\rho_0:{num_magnets}')[2]

\rho_2

In [35]:
namefunc = lambda u: f'{u.name}_mag\,0:{num_magnets}'
names = ' '.join([namefunc(u) for u in Sph.U])

magnet_positions = np.array(sp.symbols(names)).reshape(-1,num_magnets).T

In [36]:
namerange = lambda name: f'{name}_mag\,0:{num_magnets}'
names = ' '.join([namerange(u.name) for u in Sph.U])

polarities = np.array(sp.symbols(namerange('\\rho')))
magnet_positions = np.array(sp.symbols(names)).reshape(-1,num_magnets).T
magnet_positions[3,2]

φ_mag,3

In [37]:

from_v = None
to_v = magnet_positions[1]
if from_v:
    from_pos = Sph.position_vector.subs(zip(Sph.U, from_v))
else:
    from_pos = Sph.position_vector
to_pos = Sph.position_vector.subs(zip(Sph.U, to_v))
distance = sp.simplify((to_pos - from_pos).magnitude())
distance

sqrt(r_mag,1**2 - 2*r_mag,1*r(t)*sin(θ_mag,1)*sin(θ(t))*cos(φ_mag,1 - φ(t)) - 2*r_mag,1*r(t)*cos(θ_mag,1)*cos(θ(t)) + r(t)**2)

In [38]:
magnetic_energy_of = lambda magnet_pos: 1/(Sph.get_distance(magnet_pos))**3 * sp.Rational(1,3)
magnetic_energy_of(magnet_positions[0])

1/(3*(r_mag,0**2 - 2*r_mag,0*r(t)*sin(θ_mag,0)*sin(θ(t))*cos(φ_mag,0 - φ(t)) - 2*r_mag,0*r(t)*cos(θ_mag,0)*cos(θ(t)) + r(t)**2)**(3/2))

In [39]:
[magnetic_energy_of(pos) for pos in magnet_positions]

[1/(3*(r_mag,0**2 - 2*r_mag,0*r(t)*sin(θ_mag,0)*sin(θ(t))*cos(φ_mag,0 - φ(t)) - 2*r_mag,0*r(t)*cos(θ_mag,0)*cos(θ(t)) + r(t)**2)**(3/2)),
 1/(3*(r_mag,1**2 - 2*r_mag,1*r(t)*sin(θ_mag,1)*sin(θ(t))*cos(φ_mag,1 - φ(t)) - 2*r_mag,1*r(t)*cos(θ_mag,1)*cos(θ(t)) + r(t)**2)**(3/2)),
 1/(3*(r_mag,2**2 - 2*r_mag,2*r(t)*sin(θ_mag,2)*sin(θ(t))*cos(φ_mag,2 - φ(t)) - 2*r_mag,2*r(t)*cos(θ_mag,2)*cos(θ(t)) + r(t)**2)**(3/2)),
 1/(3*(r_mag,3**2 - 2*r_mag,3*r(t)*sin(θ_mag,3)*sin(θ(t))*cos(φ_mag,3 - φ(t)) - 2*r_mag,3*r(t)*cos(θ_mag,3)*cos(θ(t)) + r(t)**2)**(3/2))]

In [40]:
# get_distance(magnet_positions[1])

In [41]:
Sph.position_vector

(r(t)*sin(θ(t))*cos(φ(t)))*C.i + (r(t)*sin(θ(t))*sin(φ(t)))*C.j + (-r(t)*cos(θ(t)))*C.k

In [42]:
polarity = sp.symbols('\\rho')
magnet_position = sp.symbols(' '.join([f'{u.name}_mag' for u in Sph.U]))
energy_equation = polarity/(Sph.get_distance(magnet_position))**3 * sp.Rational(1,3)
energy_equation

\rho/(3*(r_mag**2 - 2*r_mag*r(t)*sin(θ_mag)*sin(θ(t))*cos(φ_mag - φ(t)) - 2*r_mag*r(t)*cos(θ_mag)*cos(θ(t)) + r(t)**2)**(3/2))

In [43]:
p.magnetic_potential_ASSUMPTION_single().E

\rho/(3*(r_mag**2 - 2*r_mag*r(t)*sin(θ_mag)*sin(θ(t))*cos(φ_mag - φ(t)) - 2*r_mag*r(t)*cos(θ_mag)*cos(θ(t)) + r(t)**2)**(3/2))

In [68]:
total = 0
for i in range(num_magnets):
    polarity_i = sp.symbols(f'\\rho_{i}')
    magnet_position_i = sp.symbols(' '.join([f'{u.name}_mag\,{i}' for u in Sph.U]))
    term = p.magnetic_potential_ASSUMPTION_single().E.subs(zip(magnet_position, magnet_position_i)).subs(polarity, polarity_i)
    total += sp.simplify(term.subs(dynamicsymbols('r'), sp.symbols('r')))
# total = sp.simplify(total.subs(dynamicsymbols('r'), sp.symbols('r')))
final_force = p.Energy(total).to_force().F
# 
# p.magnetic_potential_ASSUMPTION_single().E.free_symbols

In [61]:
from sympy.physics.vector import dynamicsymbols
final_force[0].subs(dynamicsymbols('r'), sp.symbols('r'))

-\rho_0*(-3*r + 3*r_mag,0*sin(θ_mag,0)*sin(θ(t))*cos(φ_mag,0 - φ(t)) + 3*r_mag,0*cos(θ_mag,0)*cos(θ(t)))/(3*(r**2 - 2*r*r_mag,0*sin(θ_mag,0)*sin(θ(t))*cos(φ_mag,0 - φ(t)) - 2*r*r_mag,0*cos(θ_mag,0)*cos(θ(t)) + r_mag,0**2)**(5/2)) - \rho_1*(-3*r + 3*r_mag,1*sin(θ_mag,1)*sin(θ(t))*cos(φ_mag,1 - φ(t)) + 3*r_mag,1*cos(θ_mag,1)*cos(θ(t)))/(3*(r**2 - 2*r*r_mag,1*sin(θ_mag,1)*sin(θ(t))*cos(φ_mag,1 - φ(t)) - 2*r*r_mag,1*cos(θ_mag,1)*cos(θ(t)) + r_mag,1**2)**(5/2)) - \rho_2*(-3*r + 3*r_mag,2*sin(θ_mag,2)*sin(θ(t))*cos(φ_mag,2 - φ(t)) + 3*r_mag,2*cos(θ_mag,2)*cos(θ(t)))/(3*(r**2 - 2*r*r_mag,2*sin(θ_mag,2)*sin(θ(t))*cos(φ_mag,2 - φ(t)) - 2*r*r_mag,2*cos(θ_mag,2)*cos(θ(t)) + r_mag,2**2)**(5/2)) - \rho_3*(-3*r + 3*r_mag,3*sin(θ_mag,3)*sin(θ(t))*cos(φ_mag,3 - φ(t)) + 3*r_mag,3*cos(θ_mag,3)*cos(θ(t)))/(3*(r**2 - 2*r*r_mag,3*sin(θ_mag,3)*sin(θ(t))*cos(φ_mag,3 - φ(t)) - 2*r*r_mag,3*cos(θ_mag,3)*cos(θ(t)) + r_mag,3**2)**(5/2))

In [69]:
final_force[1]

(-\rho_0*(3*r*r_mag,0*sin(θ_mag,0)*cos(φ_mag,0 - φ(t))*cos(θ(t)) - 3*r*r_mag,0*sin(θ(t))*cos(θ_mag,0))/(3*(r**2 - 2*r*r_mag,0*sin(θ_mag,0)*sin(θ(t))*cos(φ_mag,0 - φ(t)) - 2*r*r_mag,0*cos(θ_mag,0)*cos(θ(t)) + r_mag,0**2)**(5/2)) - \rho_1*(3*r*r_mag,1*sin(θ_mag,1)*cos(φ_mag,1 - φ(t))*cos(θ(t)) - 3*r*r_mag,1*sin(θ(t))*cos(θ_mag,1))/(3*(r**2 - 2*r*r_mag,1*sin(θ_mag,1)*sin(θ(t))*cos(φ_mag,1 - φ(t)) - 2*r*r_mag,1*cos(θ_mag,1)*cos(θ(t)) + r_mag,1**2)**(5/2)) - \rho_2*(3*r*r_mag,2*sin(θ_mag,2)*cos(φ_mag,2 - φ(t))*cos(θ(t)) - 3*r*r_mag,2*sin(θ(t))*cos(θ_mag,2))/(3*(r**2 - 2*r*r_mag,2*sin(θ_mag,2)*sin(θ(t))*cos(φ_mag,2 - φ(t)) - 2*r*r_mag,2*cos(θ_mag,2)*cos(θ(t)) + r_mag,2**2)**(5/2)) - \rho_3*(3*r*r_mag,3*sin(θ_mag,3)*cos(φ_mag,3 - φ(t))*cos(θ(t)) - 3*r*r_mag,3*sin(θ(t))*cos(θ_mag,3))/(3*(r**2 - 2*r*r_mag,3*sin(θ_mag,3)*sin(θ(t))*cos(φ_mag,3 - φ(t)) - 2*r*r_mag,3*cos(θ_mag,3)*cos(θ(t)) + r_mag,3**2)**(5/2)))/r(t)